In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [9]:
metrics_to_analyze = ["frame_count", "ear", "mar", "yaw", "pitch", "nose_tip_y"]
df = pd.read_csv("Trueimages_adrian_yarify.csv")[metrics_to_analyze].iloc[600:]
df_nonsleepy = pd.read_csv("NTHUDDD_dataset/034_noglasses_nonsleepy.csv")[metrics_to_analyze]

In [10]:
var = "ear"
mean_ear = df["ear"].mean()
fig = px.line(df, x="frame_count", y=var, title=f"Evolucion del {var} durante el video")
fig.update_layout(shapes=[
dict(
  type= 'line',
  yref= 'y', y0=mean_ear, y1=mean_ear,   # adding a horizontal line at Y = 1
  xref= 'paper', x0=0, x1=1
     ) 
])
fig.show()
mean_ear_nonsleepy = df_nonsleepy["ear"].mean()
fig = px.line(df_nonsleepy, x="frame_count", y=var, title=f"Evolucion del {var} durante el video")
fig.update_layout(shapes=[
dict(
  type= 'line',
  yref= 'y', y0=mean_ear_nonsleepy, y1=mean_ear_nonsleepy,   # adding a horizontal line at Y = 1
  xref= 'paper', x0=0, x1=1
     ) 
])
fig.show()